## 10.1 MNIST를 RNN으로

RNN(Recurrent Neural Network) : 순환 신경망 -> 순서가 있는 데이터를 처리하는데 유리 (다른 NN은 상태가 고정된 데이터 처리)

-> 앞이나 뒤의 정보에 따라 전체 의미가 달라지거나, 앞의 정보로 다음에 나올 정보를 추측하는 경우에 좋음!

* MNIST 를 RNN 방식으로 학습하고 예측!

- RNN의 가운데 한 덩어리의 신경망 : cell

-> 이 셀을 여러 개 중첩하여 deep NN 만듦 => 앞 단계에서 학습한 결과를 다음 단계의 학습에서 이용

-> 학습 데이터를 단계별로 구분해서 입력해 주어야 함! -> MNIST 입력값도 단계별로 입력할 수 있는 형태로 바꾸어 줘야!

- MNIST 데이터 : 28 X 28 -> 가로 한 줄의 28 픽셀을 한 단계의 입력값으로 사용 & 세로줄이 총 28개 -> 28단계에 걸쳐 데이터를 입력 받음

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)

learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

C:\Users\soual\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

입력값 X 앞에 n_step 차원이 추가되어 있음! -> RNN은 순서가 있는 데이터를 다룸 -> 한 번에 입력받을 갯수와 총 몇 단계로 이루어진 데이터를 받을 지 설정해야 함!

=> 가로 픽셀 수 : n_input, 세로 픽셀 수 : n_step(입력단계) 로 설정!

=> 출력값 : MNIST 의 분류 0~9까지 10개!

In [2]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

n_hidden 개의 출력값을 갖는 RNN cell 생성

위의 경우 BasicRNNCell 함수 사용 but tensorflow에 BasicLSTMCell, GRUCell 등의 다양한 방식의 셀 사용 가능

* RNN의 기본 신경망 : 긴 단계의 데이터를 학습할 경우 맨 뒤에서는 맨 앞의 정보를 잘 기억하지 못하는 특성 있음!

-> 이를 보완하기 위해 LSTM(Long Short-Term Memory), GRU(Gated Recurrent Units) 등이 만들어짐 / GRU : LSTM과 비슷 but 구조가 조금 더 간단

In [3]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)

앞에서 생성한 RNN cell 값과 입력값, 입력값의 자료형을 넣어주면 신경망 완성!

-> 원래는 한 단계 학습한 뒤 상태 저장 -> 다음 단계 입력상태로 대입하여 다시 학습 -> ... 반복

-> 고려할 것이 많음 -> 이 과정을 대신 해주는 dynamic_rnn 함수 사용!

In [4]:
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

RNN에서 나온 출력값을 갖고 최종 출력값을 만듦 -> one-hot encoding 형태로 만들 것임 -> 손실함수 : tf.nn.softmax_cross_entropy_with_logits_v2사용 => 사용하기 위해 최종 결과값이 실측값Y와 동일해야함! => [batch_size, n_class]